In [ ]:
import igraph
import matplotlib.pyplot as plt
import numpy as np
from os import path
import pandas as pd
import seaborn as sns
from selfregulation.utils.utils import get_behav_data
from sklearn.decomposition import FactorAnalysis
from sklearn.model_selection import KFold
from selfregulation.utils.r_to_py_utils import GPArotation, missForest, psychFA
from fancyimpute import SoftImpute
%matplotlib inline

# Calculate best number of components based on CV ML (SKlearn)

### Load Data

Load both cleaned and imputed data. Set up a helper function to quickly impute data within the Kfold split. Can't use data where imputation was performed over the whole dataset as there will be data leakage between training and test.

In [ ]:
# using for speed, for the time being
def SoftImpute_df(data):
    imputed_mat = SoftImpute(verbose=False).complete(data)
    return pd.DataFrame(data=imputed_mat, columns=data.columns, index=data.index)

In [ ]:
# load data
data = get_behav_data(file = 'taskdata_clean.csv', full_dataset = True)
# imputed data
imputed_data = get_behav_data(file = 'meaningful_variables_imputed.csv')
imputed_task_data = get_behav_data(file = 'taskdata_imputed.csv')
imputed_survey_data = imputed_data.drop(imputed_task_data.columns, axis = 1)
imputed_datasets = {'survey': imputed_survey_data, 'task': imputed_task_data, 'complete': imputed_data}
# cleaned data 
cleaned_data = get_behav_data( file = 'meaningful_variables_clean.csv')
cleaned_task_data = get_behav_data( file = 'taskdata_clean.csv')
cleaned_survey_data = cleaned_data.drop(cleaned_task_data.columns, axis = 1)
cleaned_datasets = {'survey': cleaned_survey_data, 'task': cleaned_task_data, 'complete': cleaned_data}

results = {'survey': {}, 'task': {}, 'complete': {}}

### Determine optimal number of factors
Two methods

Sklearn using cross-validated prediction
R's Psych package looking for best fitting model across all the data, penalized for complexity (BIC)

In [ ]:
def CV_optimal_components(data, cv):
    # find best number of components
    n_components = range(1,12)
    best_c = 0
    component_scores = []
    for c in n_components:
        fa=FactorAnalysis(c)
        scores = []
        # cross-validate fit across different train/test splits
        for train_index, test_index in kf.split(data.values):
            data_train, data_test = data.iloc[train_index], \
                                    data.iloc[test_index]
            # Impute (replace with missForest later)
            imputed_train = SoftImpute_df(data_train)
            imputed_test = SoftImpute_df(data_test)
            # fit model on training
            fa.fit(imputed_train)
            # score on test set
            scores.append(fa.score(imputed_test))
        #average score across kfolds
        score = np.mean(scores)
        component_scores.append(score)
    best_c = np.argmax(component_scores)
    print('Best Component: ', best_c)
    return best_c, component_scores

kf = KFold(n_splits = 4)
for label,data in cleaned_datasets.items():
    print('Dataset: %s' % label)
    best_c, scores= CV_optimal_components(data, kf)
    results[label]['sklearn_best_c'] = best_c
    results[label]['CV_scores'] = scores
    
for score in [d['CV_scores'] for d in results.values()]:
    plt.plot(range(len(score)),score, '-o')
plt.legend(results.keys(), loc = 'best')
plt.xlabel('# Components')
plt.ylabel('Score')

### Calculate best number of components using BIC (R Psych package)

In [ ]:
def BIC_optimal_components(data):
    BICs = []
    outputs = []
    n_components = range(1,12)
    for c in n_components:
        fa, output = psychFA(data, c)
        BICs.append(output['BIC'])
        outputs.append(output)
    best_c = np.argmin(BICs)
    print('Best Component: ', best_c)
    return best_c, BICs

for label,data in imputed_datasets.items():
    print('Dataset: %s' % label)
    best_c, BICs = BIC_optimal_components(data)
    results[label]['psych_best_c'] = best_c
    results[label]['psych_BICs'] = BICs
    
for bic in [d['psych_BICs'] for d in results.values()]:
    plt.plot(range(len(bic)),bic, '-o')
plt.legend(results.keys(), loc = 'best')
plt.xlabel('# Components')
plt.ylabel('Score')

# Interpret Task Factor Analysis Solution
Above we found that the best number of components were as follows:

**SKlearn**:

Task: {{results['task']['sklearn_best_c']}}

Survey: {{results['survey']['sklearn_best_c']}}

Complete: {{results['complete']['sklearn_best_c']}}

**Psych**:

Task: {{results['task']['psych_best_c']}}

Survey: {{results['survey']['psych_best_c']}}

Complete: {{results['complete']['psych_best_c']}}

Because psych performs rotation, is more widely used, we will use that as our main analysis. All analyses will be repeated with the sklearn pipeline as well. First we will look at the best number of components, then we will look at the solution at each dimensionality and plot the relationships between dimensions

### Interpret best components

In [ ]:
task_results = results['task']
best_c = task_results['psych_best_c']
data = imputed_datasets['task']
fa, output = psychFA(data, best_c)

In [ ]:
### Print top variables for each factor
def print_top_factors(fa_output, n = 4):
    # number of variables to display
    loading_df = pd.DataFrame(fa_output['loadings'], index=data.columns)
    for i,column in loading_df.iteritems():
        sort_index = np.argsort(abs(column))[::-1] # descending order
        top_vars = column[sort_index][0:n]
        print('\nFACTOR %s' % i)
        print(top_vars)
    return loading_df
        
loading_df = print_top_factors(output, 20)

### Histogram of factor loadings and plot of histogram relationship
We can see that factors 4 and 5 are really just pulling from a few variables. They are really "task" factors - tapping discounting tasks and the ART task respectively. The CCT Hot "number of cards chosen" maps onto factor 5, but to a far smaller extent

In [ ]:
sns.set_context('talk')
loading_df.hist()

The factors are roughly orthogonal. Orthogonality wasn't enforced, and factors were rotated using "oblimin"

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(loading_df.corr())

### Interpretation/Naming of Factors 

Factor 0: Speed of Processing (Drift)

Factor 1: Memory/Abstraction

Factor 2: Stimulus Encoding Time (Non-Decision)

Factor 3: Response Caution (Threshold)

Factor 4: Dicount Rate

Factor 5: Risk Taking

## Do the same for all other dimensionalities

In [ ]:
factor_names = {}

**One Factor**

Factor 0: Cognitive Ability

In [ ]:
fa, output = psychFA(data, 1)
loading_df = print_top_factors(output)
factor_names[1] = ['CogAblty']

**Two Factors**

Factor 0: Speed of Processing (Drift)

Factor 1: Discount Rate

In [ ]:
fa, output = psychFA(data, 2)
loading_df = print_top_factors(output, 6)
factor_names[2] = ['Drift','Discount']

**Three Factors**

Factor 0: Speed of Processing (Drift)

Factor 1: Encoding Time (Non-Decision)

Factor 2: Discount Rate

In [ ]:
fa, output = psychFA(data, 3)
loading_df = print_top_factors(output, 6)
factor_names[3] = ['Drift','Non-Dec','Discount']

**Four Factors**

Factor 0: Speed of Processing (Drift)

Factor 1: Memory/Abstraction

Factor 2: Encoding Time (Non-Decision)

Factor 3: Discount Rate

In [ ]:
fa, output = psychFA(data, 4)
loading_df = print_top_factors(output, 6)
factor_names[4] = ['Drift','Mem','Non-Dec','Discount']

**Five Factors**

Factor 0: Speed of Processing (Drfit)

Factor 1: Memory/Abstraction 

Factor 2: Encoding Time (Non-Decision)

Factor 3: Response Caution (Threshold)

Factor 4: Discount Rate

In [ ]:
fa, output = psychFA(data, 5)
loading_df = print_top_factors(output, 6)
factor_names[5] = ['Drift','Mem','Non-Dec','Thresh','Discount']

**Six Factors**

Factor 0: Speed of Processing (Drift)

Factor 1: Memory/Abstraction

Factor 2: Stimulus Encoding Time (Non-Decision)

Factor 3: Response Caution (Threshold)

Factor 4: Dicount Rate

Factor 5: Risk Taking

In [ ]:
fa, output = psychFA(data, 6)
loading_df = print_top_factors(output)
factor_names[6] = ['Drift','Mem','Non-Dec','Thresh','Discount','Risk']

**Seven Factors**

Factor 0: Speed of Processing (Drift)

Factor 1: Information Use

Factor 2: Stimulus Encoding Time (Non-Decision)

Factor 3: Dicount Rate

Factor 4: Response Caution (Threshold)

Factor 5: Response Caution (Speed of action)

Factor 6: Risk Taking

In [ ]:
fa, output = psychFA(data, 7)
loading_df = print_top_factors(output, 8)
factor_names[7] = ['Drift','Inf-Use','Non-Dec','Discount','Thresh','Speed', 'Risk']

**Eight Factors**

Factor 0: Speed of Processing (Drift)

Factor 1: Memory/Abstraction

Factor 2: Stimulus Encoding Time (Non-Decision)

Factor 3: Dicount Rate

Factor 4: Response Caution (Threshold)

Factor 5: Risk Taking

Factor 6: Information Use (CCT information)

Factor 7: Response Caution (Speed of action on IST and TOL)


In [ ]:
fa, output = psychFA(data, 8)
loading_df = print_top_factors(output, 6)
factor_names[8] = ['Drift','Mem','Non-Dec','Discount','Thresh','Risk','Inf-Use','Speed']

### Plot relationships between solutions at different dimensionalities

Each factor analytic solution gives us some components. By correlating the components at one level with the next, we can get a sense for how similar these components are. When we move to 3 dimensions from 2, is it the case that one of the dimensions just split? Or is there a new, non-hierarchical parcellation of the space?



In [ ]:
# helper functions for plotting dimensionality relations
# compute FA with different dimensionalities and quantify similarity
def dimensional_similarity(factors, reference):
    ''' This function returns an N x M correlation matrix where N is the number
    of reference (lower dimension) factors and M is the number of higher dimension
    factors
    '''
    relation = np.corrcoef(reference.T, factors.T)
    # each row is a reference factor, each column a new factor
    relation=relation[:reference.shape[1], reference.shape[1]:]
    return relation

def construct_relational_tree(similarities, labels=None, filey=None):
    ''' Takes a list of similarities and constructs a tree graph, then plots
    '''
    G = igraph.Graph()
    layer_start = 0
    colors = ['red','blue','green','violet']*4
    for similarity in similarities:
        curr_color = colors.pop()
        origin_length = similarity.shape[0]
        target_length = similarity.shape[1]
        if len(G.vs)==0:
            G.add_vertices(origin_length)
        G.add_vertices(target_length)
        for i in range(origin_length):
            for j in range(target_length):
                G.add_edge(i+layer_start,j+origin_length+layer_start,
                           weight=abs(similarity[i,j]*5),color = curr_color)
        layer_start+=similarity.shape[0]
    layout = G.layout_reingold_tilford(root = [0])
    if labels == None:
        labels = G.vs.indices
    plot = igraph.plot(G, layout=layout, **{'inline': True, 'vertex_label': labels, 
                                            'vertex_color': 'white',
                                            'vertex_size': 50, 
                                         'edge_width':[w for w in G.es['weight']], 
                                         'edge_color': G.es['color'], 
                                            'bbox': (1000,1000), 'margin':100})
    if filey:
        plot.save(filey)
    #G.write_dot('test.dot')
    return G


In [ ]:
# construct array of component similarities
similarities = []
components = np.sort(list(factor_names.keys()))
reference = None
for c in components:
    fa, output = psychFA(data,c)
    result = pd.DataFrame(output['scores'], data.index)
    if reference is not None:
        similarity = dimensional_similarity(result,reference)
        similarities.append(similarity)
    reference = result




In [ ]:
# plot component similarity tree
labels = []
for c in components:
    labels+=factor_names[c]
similarity_tree = construct_relational_tree(similarities, filey=path.join('Plots', 'task_graph.png'))

# Scrap

In [ ]:
for label,data in imputed_datasets.items():
    best_c = task_data['sklearn_best_c']
    # construct array of component similarities
    similarities = []
    components = range(1,best_c+2)
    reference = None
    for c in components:
        fa=FactorAnalysis(c)
        result = pd.DataFrame(fa.fit_transform(data.values), data.index)
        if c>1:
            result = GPArotation(result, method='oblimin')
        if reference is not None:
            similarity = dimensional_similarity(result,reference)
            similarities.append(similarity)
        reference = result

    # plot tree
    labels = []
    for c in np.sort(list(factor_names.keys())):
        labels+=factor_names[c]
    similarity_tree = construct_relational_tree(similarities, filey=path.join('Plots', + label +'_graph.png'))


In [ ]:
"""
# Take max for each dimension?
binarized_similarities = []
for s in similarities:
    r,c = s.shape
    binarized_similarities.append(s==np.resize(s.max(axis=1),(c,r)).T)
"""
thresh = .5
binarized_similarities = []
for s in similarities:
    binarized_similarities.append((s>thresh)*1)

## Visualization of optimal factor structure

### Sum over task variables (extrememly questionable) and then plot the relationships across tasks

In [ ]:
from plot_utils import dendroheatmap_left
# *****************************************************************************
# sum over tasks
# *****************************************************************************
tasks = np.unique([i.split('.')[0] for i in data.columns])
task_sums = {}
for task in tasks:  
    task_sums[task] = result.filter(regex=task,axis=0).sum()
task_sums = pd.DataFrame(task_sums).T
dendroheatmap = dendroheatmap_left(task_sums.T.corr())

### multidimensional scaling

In [ ]:
# *****************************************************************************
# visualize the similarity of the measurements in FA space
# *****************************************************************************

from data_preparation_utils import convert_var_names
from graph_utils import distcorr_mat
from sklearn import manifold
from sklearn.metrics import euclidean_distances
import seaborn as sns

seed = np.random.RandomState(seed=3)
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)

tsne = manifold.TSNE(n_components=2, random_state=seed, metric="precomputed")

# compute distances between variables
# using distance correlation
# result_distances = 1-distcorr_mat(result.T.values)
# euclidean
result_distances = euclidean_distances(result)
# transform
mds_transform = mds.fit_transform(result_distances)
tsne_transform = tsne.fit_transform(result_distances)

# plot
tasks = [i.split('.')[0] for i in result.index]
colors = sns.color_palette("husl", len(np.unique(tasks)))

fig, ax = sns.plt.subplots(figsize = (20,20))
ax.scatter(mds_transform[:,0], mds_transform[:,1])

variables = convert_var_names(list(result.index))
for i, txt in enumerate(variables):
    ax.annotate(txt, (mds_transform[i,0],mds_transform[i,1]), size = 15)
